In [1]:
import re

%matplotlib inline
import matplotlib
import seaborn as sns; sns.set()

datapath = "data/neko.txt.cabocha"

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

### 準備
 ```
 brew install cabocha
 cabocha -f1 neko.txt > neko.txt.cabocha
 ```
 
### memo
クラスに`__repr__`を定義するとprint等で呼ばれた時の表示方法を指定できる


In [2]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def __repr__(self):
        return self.surface

In [3]:
prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")

def gen_sentence():
    morph_list = []
    with open(datapath) as f:
        for line in f:
            res_m = prg_morph.match(line)
            if res_m:
                morph_list.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            elif line == "EOS\n":
                yield morph_list
                morph_list = []

for i, sentence in enumerate(gen_sentence()):
    print(sentence)
    if i > 10:
        break
            

[一]
[]
[　, 吾輩, は, 猫, で, ある, 。]
[名前, は, まだ, 無い, 。]
[]
[　, どこ, で, 生れ, た, か, とんと, 見当, が, つか, ぬ, 。]
[何, でも, 薄暗い, じめじめ, し, た, 所, で, ニャーニャー, 泣い, て, いた事, だけ, は, 記憶, し, て, いる, 。]
[吾輩, は, ここ, で, 始め, て, 人間, という, もの, を, 見, た, 。]
[しかも, あと, で, 聞く, と, それ, は, 書生, という, 人間, 中, で, 一番, 獰悪, な, 種族, で, あっ, た, そう, だ, 。]
[この, 書生, という, の, は, 時々, 我々, を, 捕え, て, 煮, て, 食う, という, 話, で, ある, 。]
[しかし, その, 当時, は, 何, という, 考, も, なかっ, た, から, 別段, 恐し, いとも, 思わ, なかっ, た, 。]
[ただ, 彼, の, 掌, に, 載せ, られ, て, スー, と, 持ち上げ, られ, た, 時, 何だか, フワフワ, し, た, 感じ, が, あっ, た, ばかり, で, ある, 。]


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

### memo
文節、形態素、文末の判定は正規表現でやる

1. 文節

    - 形態素は以降の行に書いてあるので後でappendするための受け皿をつくっておく。
    - 係り元を覚えておくための辞書を作る。日本語は係り先の方が文の後ろにあるから先読みせず1行ずつ処理可能
    - EOSが2行以上連なってるところが1文にカウントされてしまっている
    
2. 形態素

    4章を丸々流用

In [4]:
class Chunk:
    def __init__(self, dst, srcs):
        self.morphs = []
        self.dst = dst
        self.srcs = srcs
    
    def __repr__(self):
        "形態素をくっつけたものを表示"
        return ''.join(map(lambda m:m.__repr__(), self.morphs))
    
    def append(self, morph):
        "文節に形態素を追加"
        self.morphs.append(morph)

In [37]:
from collections import defaultdict

prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")
prg_chunk = re.compile(r"\* (?P<num>\d+) (?P<dst>-?\d+)D \d+/\d+ .*")

def gen_sentence():
    chunk = Chunk(-1, []) # 名前確保用
    sentence, chunk_srcs = [], defaultdict(list)
    with open(datapath) as f:
        for line in f:
            res_m = prg_morph.match(line)
            res_c = prg_chunk.match(line)
            if res_c:
                me, dst = int(res_c.group("num")), int(res_c.group("dst"))
                chunk_srcs[dst].append(me)
                chunk = Chunk(dst, chunk_srcs[me])
                sentence.append(chunk)
            elif res_m:
                chunk.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            else: # End of sentence
                yield sentence
                sentence, chunk_srcs = [], defaultdict(list)

def dependency_parsing(n):
    counter = 0
    for sentence in gen_sentence():
        if sentence:
            counter += 1
            if counter == n:
                print("sentence",counter)
                for chunk in sentence:
                    print(chunk, "\t->", sentence[chunk.dst] if chunk.dst != -1 else None)

dependency_parsing(8)
            

sentence 8
この 	-> 書生というのは
書生というのは 	-> 話である。
時々 	-> 捕えて
我々を 	-> 捕えて
捕えて 	-> 煮て
煮て 	-> 食うという
食うという 	-> 話である。
話である。 	-> None


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．



In [39]:
for sentence in gen_sentence():
    for chunk in sentence:
        if ("名詞" in list(map(lambda m: m.pos, chunk.morphs))) and "動詞" in list(map(lambda m: m.pos, sentence[chunk.dst].morphs)):
            print(chunk, "\t", sentence[chunk.dst]) if chunk.dst != -1 else None

　どこで 	 生れたか
見当が 	 つかぬ。
所で 	 泣いて
ニャーニャー 	 泣いて
いた事だけは 	 記憶している。
吾輩は 	 見た。
ここで 	 始めて
ものを 	 見た。
あとで 	 聞くと
我々を 	 捕えて
掌に 	 載せられて
スーと 	 持ち上げられた
時 	 フワフワした
感じが 	 あったばかりである。
上で 	 落ちついて
顔を 	 見たのが
ものの 	 見始であろう。
ものだと 	 思った
感じが 	 残っている。
今でも 	 残っている。
第一毛をもって 	 装飾されべきはずの
顔が 	 つるつるして
その後 	 逢ったが
猫にも 	 逢ったが
一度も 	 出会わした
真中が 	 突起している。
中から 	 吹く。
ぷうぷうと煙を 	 吹く。
咽せぽくて 	 弱った。
人間の 	 飲む
事は 	 知った。
頃 	 知った。
裏で 	 坐って
心持に 	 坐って
速力で 	 運転し始めた。
書生が 	 動くのか
動くのか 	 動くのか
自分だけが 	 動くのか
動くのか 	 分らないが
眼が 	 廻る。
胸が 	 悪くなる。
音が 	 して
眼から 	 出た。
火が 	 出た。
それまでは 	 記憶しているが
記憶しているが 	 分らない。
あとは 	 分らない。
事やら 	 分らない。
気が 	 付いて
書生は 	 いない。
たくさん 	 おった
兄弟が 	 見えぬ。
一疋も 	 見えぬ。
母親さえ 	 隠してしまった。
姿を 	 隠してしまった。
所とは 	 違って
眼を 	 明いていられぬくらいだ。
吾輩は 	 棄てられたのである。
上から 	 棄てられたのである。
急に 	 棄てられたのである。
中へ 	 棄てられたのである。
思いで 	 這い出すと
笹原を 	 這い出すと
向うに 	 ある。
池が 	 ある。
吾輩は 	 見た。
前に 	 坐って
分別も 	 出ない。
書生がまた 	 来てくれるかと
迎に 	 来てくれるかと
試みに 	 やって
誰も 	 来ない。
上を 	 渡って
風が 	 渡って
日が 	 かかる。
暮れ 	 かかる。
腹が 	 減って来た。
非常に 	 減って来た。
声が 	 出ない。
何でも 	 廻り始めた。
食物の 	 ある
所まで 	 ある
決心を 	 して
池を 	 廻り始めた。
左りに 	 廻り始

吾輩に 	 向って
下のごとく 	 質問した。
「御めえは 	 ある」
今までに 	 とった
鼠を 	 とった
何匹 	 とった
事が 	 ある」
智識は 	 発達しているつもりだが
黒よりも 	 発達しているつもりだが
発達しているつもりだが 	 ならないと
勇気とに 	 至っては
比較には 	 ならないと
覚悟は 	 していた
問に 	 接したる
事実は 	 行かないから、
訳には 	 行かないから、
吾輩は 	 答えた。
黒は 	 笑った。
先から 	 張っている
突 	 張っている
髭を 	 震わせて
非常に 	 笑った。
黒は 	 あって、
自慢を 	 する
丈に 	 足りない
どこか 	 足りない
ところが 	 あって、
気焔を 	 感心したように
感心したように 	 謹聴していれば
咽喉を 	 鳴らして
謹聴していれば 	 御しやすい
吾輩は 	 定めた。
近付に 	 なってから
呼吸を 	 飲み込んだから
場合にも 	 わるくするのも
己れを 	 弁護して
弁護して 	 わるくするのも
形勢を 	 わるくするのも
事彼に 	 しゃべらして
手柄話を 	 しゃべらして
御茶を 	 濁すに
思案を 	 定めた。
「君などは 	 とったろう」と
年であるから 	 とったろう」と
彼は 	 吶喊して来た。
欠所に 	 吶喊して来た。
三四十は 	 とったろう」とは
彼は 	 合わねえ。
語を 	 つづけて
二百は 	 引き受けるが
一人で 	 引き受けるが
いつでも 	 引き受けるが
いたちってえ 	 合わねえ。
奴は 	 合わねえ。
手に 	 合わねえ。
いたちに 	 向って
目に 	 逢った」
相槌を 	 打つ。
黒は 	 云う。
眼を 	 ぱちつかせて
亭主が 	 持って
袋を 	 持って
下へ 	 這い込んだら
御めえ大きな 	 いたちの
野郎が 	 喰って
面 	 喰って
感心して 	 見せる。
こん畜生って 	 追っかけて
気で 	 追っかけて
中へ 	 追い込んだと
御めえいざってえ段に 	 なると
奴め最後っ屁を 	 こきゃがった。
いたちを 	 見ると
彼は 	 なで廻わした。
ここに 	 至って
臭気を 	 感ずるごとく
今なお 	 感ずるごとく
前足を 	 揚げて
頭を 	 なで廻わした。
二三遍 	 なで廻わした。
吾輩も 	 する。

女が 	 有りそうな、
面白そうな、 	 並べては
文句ばかり 	 並べては
人間を 	 求めて、
来るのから 	 して
合点が 	 行かぬが、
牡蠣的主人が 	 聞いて
談話を 	 聞いて
相槌を 	 打つのは
御無沙汰を 	 しました。
暮から 	 活動している
大に 	 活動している
ものですから、 	 向かないので」と
方角へ 	 向かないので」と
足が 	 向かないので」と
紐を 	 ひねくりながら
謎 	 見たような
事を 	 いう。
方角へ 	 向く
足が 	 向く
かね」と 	 引張る。
主人は 	 引張る。
顔を 	 して、
袖口を 	 引張る。
羽織は 	 はみ出している。
下から 	 はみ出している。
べら者が 	 はみ出している。
左右へ 	 はみ出している。
五分くらいずつ 	 はみ出している。
「エヘヘヘ少し 	 違った
方角で」と 	 笑う。
寒月君が 	 笑う。
今日は 	 欠けている。
前歯が 	 欠けている。
一枚 	 欠けている。
「君歯を 	 どうかしたかね」と
主人は 	 転じた。
問題を 	 転じた。
所で 	 食いましてね」
椎茸を 	 食いましてね」
「何を 	 食ったって？
椎茸を 	 食ったんで。
傘を 	 噛み切ろうとしたら
前歯で 	 噛み切ろうとしたら
ぼろりと 	 欠けましたよ」
歯が 	 欠けましたよ」
「椎茸で 	 かける
前歯が 	 かける
俳句には 	 なるかも知れないが、
恋には 	 ならんようだな」と
ならんようだな」と 	 叩く。
平手で 	 叩く。
頭を 	 叩く。
例のですか、 	 肥ってるじゃありませんか、それなら
黒にだって 	 負けそうも
負けそうも 	 ありませんね、
「近頃大分 	 大きくなったのさ」と
大きくなったのさ」と 	 なぐる。
自慢そうに 	 なぐる。
頭を 	 なぐる。
「一昨夜も 	 やりましてね」と
合奏会を 	 やりましてね」と
寒月君は 	 戻す。
もとへ 	 戻す。
「どこで」 	 ならんでもよいでしょう。
「どこでも 	 ならんでもよいでしょう。
ヴァイオリンも 	 なると
三挺くらいに 	 なると
下手でも 	 聞かれる
二人は 	 まじりましたが、
女で 	 まじりましたが、
私が 	 まじりましたが、
中へ 	 まじりましたが、
自分でも 	 弾け

何でも 	 落ちるまで
魔が 	 落ちるまで
意気込みで 	 引っ掻き廻す。
無茶苦茶に 	 引っ掻き廻す。
顔中 	 引っ掻き廻す。
猛烈なので 	 倒れかかる。
中心を 	 失って
たびに 	 とらなくてはならぬから、
後足で 	 とらなくてはならぬから、
調子を 	 とらなくてはならぬから、
一つ所に 	 いる
訳にも 	 行かんので、
台所中あちら、 	 廻る。
こちらと 	 飛んで
我ながら 	 起っていられた
器用に 	 起っていられた
ものだと 	 思う。
真理が 	 現前する。
「危きに 	 臨めば
ものを 	 為し能う。
天祐という」幸に 	 戦っていると、
天祐を 	 戦っていると、
吾輩が 	 戦っていると、
魔と 	 戦っていると、
足音が 	 して
奥より 	 来るような
人が 	 来るような
ここで 	 来られては
人に 	 来られては
大変だと 	 思って、
躍起と 	 なって
台所を 	 かけ廻る。
足音は 	 近付いてくる。
残念だが 	 足りない。
天祐が 	 足りない。
小供に 	 見付けられた。
「あら猫が 	 食べて
御雑煮を 	 食べて
踊を 	 踊っている」と
声を 	 する。
声を 	 聞きつけたのが
第一に 	 聞きつけたのが
羽子板も 	 打ち遣って
勝手から 	 飛込んで来る。
細君は 	 仰せられる。
紋付で 	 仰せられる。
猫ねえ」と 	 仰せられる。
主人さえ 	 出て来て
書斎から 	 出て来て
馬鹿野郎」と 	 いった。
みんな 	 申し合せたように
申し合せたように 	 笑っている。
腹は 	 立つ、
踊は 	 弱った。
訳に 	 ゆかぬ、
笑いが 	 やみそうになったら、
やみそうになったら、 	 なる
五つに 	 なる
女の子が 	 いったので
狂瀾を 	 するという
既倒に 	 するという
勢でまた 	 笑われた。
大変 	 笑われた。
実行も 	 大分見聞したが、
時 	 感じた
天祐も 	 消え失せて、
どっかへ 	 消え失せて、
通り四つ這に 	 なって、
眼を 	 白黒するの
醜態を 	 演ずるまでに
見殺しに 	 するのも
気の毒と 	 見えて
餅を 	 とってやれ」と
主人が 	 命ずる。
御三に 	 命ずる。
御三は 	 見る。
眼付で 	 見る。
細君を 	 見る。
細君は 	

主人も 	 負けずに
「結構で」と 	 注す。
油を 	 注す。
「同志だけが 	 よりまして
朗読会というのを 	 組織しまして、
組織しまして、 	 続けたいつもりで、
一回 	 会合して
会合して 	 続けたいつもりで、
研究を 	 続けたいつもりで、
暮に 	 開いた
朗読会と 	 云うと
何か 	 附けて、
節奏でも 	 附けて、
類を 	 読むように
読むように 	 聞えますが、
風に 	 やるんです」
やるんです」 	 やるつもりです」
初めは 	 やるつもりです」
作から 	 やるつもりです」
創作なんかも 	 やるつもりです」
やるつもりです」 	 ものででもあるんですか」
作と 	 いうと
琵琶行のような 	 ものででもあるんですか」
「それじゃ、 	 やったんです」
ものを 	 やったんです」
やったんです」 	 やりました」
「せんだっては 	 やりました」
心中物を 	 やりました」
近松と 	 いえば
近松に 	 極っている。
それを 	 聞き直す
愚だと 	 思っていると、
主人は 	 撫でている。
何にも 	 分らずに
頭を 	 撫でている。
叮嚀に 	 撫でている。
藪睨みから 	 惚れられたと
人間も 	 ある
世の中だから 	 足らんと
誤謬は 	 足らんと
ままに 	 すましていた。
東風子は 	 窺う。
顔色を 	 窺う。
「それじゃ 	 朗読するのですか、または
一人で 	 朗読するのですか、または
朗読するのですか、または 	 やるんですか」
役割を 	 やるんですか」
やるんですか」 	 見ました。
「役を 	 懸
合で 	 やって
主意は 	 添えます。
人物に 	 持って
同情を 	 持って
性格を 	 発揮するのを
第一として、 	 添えます。
それに 	 添えます。
身振りを 	 添えます。
白は 	 やるんです」
人を 	 写し出すのが
主で、 	 やるんです」
丁稚でも、 	 出てきたように
人物が 	 出てきたように
出てきたように 	 やるんです」
ものですな」 	 行きますか」
「失礼ながら 	 行きますか」
第一回としては 	 成功した
方だと 	 思います」
前 	 やったと
心中物と 	 いうと」
船頭が 	 行く
御客を 	 乗せて
芳原へ 	 行く
所なんで」 	 やりましたな」と
幕を 	

...

負けに 	 なりそうだから……」
最初から 	 負けても
「僕は 	 構わないが、
君には 	 勝たしたくない」
春風影裏に 	 きってるね」
電光を 	 きってるね」
「ハハハハもう 	 なっていい
逆かに 	 なっていい
時分だと 	 思ったら、
ところが 	 あるね。
それじゃ 	 あきらめるかな」
無常迅速、 	 あきらめるさ」
「アーメン」と 	 下した。
迷亭先生今度は 	 下した。
方面へ 	 下した。
一石を 	 下した。
前で 	 争っていると、
独仙君が 	 争っていると、
一生懸命に 	 争っていると、
輸贏を 	 争っていると、
入口には、 	 坐っている。
相ならんで 	 坐っている。
傍に 	 して
主人が 	 して
顔を 	 して
前に 	 排列してあるのは
三本、 	 排列してあるのは
まま 	 排列してあるのは
上に 	 排列してあるのは
出処は 	 ぬくもっていた。
懐で、 	 取り出した
時は 	 ぬくもっていた。
手のひらに 	 感じた
裸ながら 	 ぬくもっていた。
東風君は 	 注いでいると、
眼を 	 して
視線を 	 注いでいると、
上に 	 注いでいると、
寒月君は 	 開いた。
口を 	 開いた。
四日ばかり前に 	 帰って来たのですが、
国から 	 帰って来たのですが、
用事が 	 あって、
方々馳け 	 あるいていた
ものですから、 	 上がられなかったのです」
主人は 	 云う。
例のごとく 	 云う。
事を 	 云う。
おみやげを 	 献上しないと
「名産だって 	 有りそうだぜ」と
東京にも 	 有りそうだぜ」と
そんなのは 	 有りそうだぜ」と
有りそうだぜ」と 	 見る。
主人は 	 見る。
一番 	 取り上げて、
奴を 	 取り上げて、
一本 	 取り上げて、
先へ 	 持って行って
臭いを 	 かいで
「かいだって、 	 わかりませんよ」
善悪は 	 わかりませんよ」
御覧 	 なさい」
事は 	 食べるが、
こいつは 	 欠けてるじゃないか」
先が 	 欠けてるじゃないか」
「それだから 	 持って来ないと
心配だと 	 云うのです」
鼠が 	 食ったのです」
滅多に 	 食うと
ペストに 	 なるぜ」
「なに大丈夫、 	 かじったって
害は 	 ありません」
「全体どこで 	 噛ったんだい」


男だ」と 	 ぷんぷんしている。
主人は 	 ぷんぷんしている。
人なら 	 構いませんがね、
二千でも 	 構いませんがね、
生徒が 	 して、
腕まくりを 	 して、
ステッキを 	 持って
徘徊しているんだから 	 出せませんよ。
容易に 	 出せませんよ。
手を 	 出せませんよ。
中には 	 ありますからね。
沈澱党などと 	 号して、
号して、 	 喜んでるのが
いつまでも 	 溜まって
底に 	 溜まって
喜んでるのが 	 ありますからね。
そんなのに 	 限って
滅多に 	 出来ません。
ヴァイオリンなどに 	 出来ません。
手出しは 	 出来ません。
目に 	 逢うか
ヴァイオリンを 	 弾いて
「それじゃ、 	 やめたんだね」と
やめたんだね」と 	 押す。
主人が 	 押す。
念を 	 押す。
かたを 	 つけたらよさそうな
こっちの 	 思うように
思うように 	 あく
埒が 	 あく
もんじゃありませんよ」と 	 云いながら
寒月君は 	 ふかし出した。
「朝日」へ 	 つけて
火を 	 つけて
　主人は 	 面倒になったと
面倒になったと 	 見えて、
書斎へ 	 這入ったと
洋書を 	 持ち出して来て、
一冊 	 持ち出して来て、
腹這に 	 なって
独仙君は 	 とっている。
前へ 	 退去して、
退去して、 	 とっている。
独りで 	 並べて
碁石を 	 並べて
一人相撲を 	 とっている。
逸話も 	 かかるので
聴手が 	 減り
一人 	 減り
二人 	 減って、
芸術に 	 なる。
東風君と、 	 なる。
事に 	 辟易した
迷亭先生のみと 	 なる。
煙を 	 ふうと
世の中へ 	 吹き出した
寒月君は、 	 つづける。
速度を 	 もって
談話を 	 つづける。
「東風君、 	 思ったね。
僕は 	 思ったね。
時 	 思ったね。
駄目だ、と 	 云って
真夜中に 	 来れば
金善は 	 寝てしまうから
何でも 	 帰りつくして、
生徒が 	 帰りつくして、
散歩から 	 帰りつくして、
金善が 	 寝ない
時を 	 見計らって来なければ、
計画が 	 帰する。
水泡に 	 帰する。
僕は 	 見積ったね。
時間を 	 見積ったね。
十時頃と 	 見積ったね。
十時頃まで 	 暮さなければならない。
どこかで 	 暮さな

「気が 	 しても
先生」と 	 すである。
寒月君は 	 すである。
一座を 	 見廻
わして 	 すである。
よう 	 すである。
そこまで 	 持って行くには
ものが 	 あったのだろう。
僕は 	 拝聴していたんだよ」と
サンドラ・ベロニが 	 出現する
邦に 	 出現する
ところかと 	 思って、
今が 	 拝聴していたんだよ」と
今まで 	 拝聴していたんだよ」と
真面目に 	 拝聴していたんだよ」と
拝聴していたんだよ」と 	 云った
誰か 	 聞くかと
講釈でも 	 聞くかと
何にも 	 出ないので
質問が 	 出ないので
「サンドラ・ベロニが 	 弾いて、
月下に 	 弾いて、
竪琴を 	 弾いて、
歌を 	 うたってる
中で 	 うたってる
庚申山へ 	 かかえて
ヴァイオリンを 	 かかえて
同曲に 	 して
事に 	 驚ろかし、
向うは 	 驚ろかし、
嫦娥を 	 驚ろかし、
驚ろかし、 	 おどろかされたので、
君は 	 おどろかされたので、
怪狸に 	 おどろかされたので、
ところで 	 来たした。
大差を 	 来たした。
遺憾だろう」と 	 説明すると、
一人で 	 説明すると、
「全体 	 弾こうなんて、
山の上で 	 弾こうなんて、
ヴァイオリンを 	 弾こうなんて、
ハイカラを 	 やるから、
おどかされるんだ」と 	 加えると、
今度は 	 加えると、
主人が 	 加えると、
酷評を 	 加えると、
「好漢 	 向って
鬼窟裏に 	 向って
生計を 	 営む。
事だ」と 	 嘆息した。
独仙君は 	 嘆息した。
すべて 	 云う
独仙君の 	 云う
事は 	 わかった
寒月君に 	 わかった
寒月君ばかりではない、 	 わからないだろう。
誰にでも 	 わからないだろう。
寒月君、 	 磨いてるのかね」と
近頃でも 	 磨いてるのかね」と
矢張り学校へ 	 行って
珠ばかり 	 磨いてるのかね」と
磨いてるのかね」と 	 転じた。
迷亭先生は 	 転じた。
話頭を 	 転じた。
こないだうちから 	 帰省していた
国へ 	 帰省していた
珠も 	 あきましたから、
珠が 	 磨けないと
博士には 	 なれんぜ」と
主人は 	 ひそめたが、
眉を 	 ひそめたが、
博士ならも 	 うならなくってもいいんです」
結婚が 	 延びて、

間 	 払うのだから、
十円ずつ 	 払うのだから、
払うのだから、 	 払えばいいのだ。
先方では 	 払えばいいのだ。
六十回 	 払えばいいのだ。
そこが 	 なる。
ところで、 	 なる。
六十回も 	 繰り返していると、
事を 	 繰り返していると、
毎月 	 繰り返していると、
六十一回にも 	 なる。
十円 	 払う
気に 	 なる。
六十二回にも 	 払う
十円 	 払う
気に 	 なる。
六十三回、 	 払わなくては
回を 	 重ねるに
期日が 	 くれば
十円 	 払わなくては
気が 	 済まないようになる。
利口のようだが、 	 ある。
習慣に 	 迷って、
根本を 	 忘れると
大弱点が 	 ある。
弱点に 	 乗じて
僕が 	 するのさ」
十円ずつ 	 するのさ」
毎月得を 	 するのさ」
寒月君が 	 笑うと、
事は 	 あるよ。
僕は 	 ある」と
貸費を 	 勘定せずに
毎月 	 勘定せずに
勘定せずに 	 返して、
向から 	 断わられた
事が 	 ある」と
恥を 	 公言した。
恥のように 	 公言した。
人が 	 いるから
ここに 	 いるから
先刻 	 述べた
未来記を 	 聞いて
冗談だなどと 	 笑う
六十回で 	 考える
月賦を 	 払って
生涯 	 払って
正当だと 	 考える
青年諸君は、 	 だまされないようにしなくっちゃいけない」
僕らの 	 云う
事を 	 聞いて
月賦は 	 致します」
事に 	 致します」
参考に 	 なる
寒月君」と 	 向いだした。
独仙君は 	 向いだした。
寒月君に 	 向いだした。
迷亭君が、 	 結婚したのが
君が 	 結婚したのが
無断で 	 結婚したのが
穏当でないから、 	 謝罪しろと
金田とか 	 云う
人に 	 謝罪しろと
謝罪しろと 	 忠告したら
「謝罪は 	 あずかりたいですね。
御容赦に 	 あずかりたいですね。
向うが 	 あやまるなら
方では 	 ありません」
慾は 	 ありません」
「警察が 	 あやまれと
君に 	 あやまれと
御免 	 蒙ります」
御免 	 蒙ります」
「それ 	 見た
今とは 	 変ってる。
人間が 	 変ってる。
御威光なら 	 出来た
何でも 	 出来た
御威光でも 	 出来ない
ものが 	 出来てくる
閣下でも、 	 出来ない


人に 	 紹介して上げます」
「癒らんでも 	 差支えない」
「そげん頑固張りなさるなら 	 やむを得ません。
あなたは 	 来てくれますか」
「僕かね、 	 是非行くよ。
栄を 	 得たい
そこ 	 いらだろうと
仲人が 	 出来ても
二人 	 出来ても
人間として 	 出席するよ」
寒月君は 	 出てくれるでしょうね。
関係も 	 あるから」
事に 	 します、
僕の 	 作った
曲を 	 奏するのを、
楽隊が 	 奏するのを、
奏するのを、 	 きき落すのは
前で 	 朗読したいです」
新体詩を 	 朗読したいです」
私は 	 生れてから、
一杯ビールを 	 飲みます」と
自分で 	 買って来た
ビールを 	 飲んで
一人で 	 飲んで
真赤に 	 なった。
日は 	 暮れて、
死骸が 	 乱す
算を 	 乱す
なかを 	 見れば
火は 	 消えている。
昔に 	 消えている。
連中も 	 尽きたと
興が 	 尽きたと
独仙君が 	 立ち上がる。
「僕も 	 帰る」と
口々に 	 出る。
玄関に 	 出る。
寄席が 	 はねた
あとのように 	 淋しくなった。
座敷は 	 淋しくなった。
　主人は 	 入る。
夕飯を 	 すまして
書斎に 	 入る。
妻君は 	 縫う。
襟を 	 かき合せて、
不断着を 	 縫う。
小供は 	 寝る。
枕を 	 並べて
下女は 	 行った。
湯に 	 行った。
呑気と 	 見える
人々も、 	 見ると、
底を 	 叩いて
音が 	 する。
悟ったようでも 	 踏まぬ。
足は 	 踏まぬ。
ほかは 	 踏まぬ。
絵に 	 かいた
寒月君は 	 連れて来た。
珠磨りを 	 やめて
お国から 	 連れて来た。
奥さんを 	 連れて来た。
順当が 	 続くと
東風君も 	 したら、
十年 	 したら、
新体詩を 	 捧げる
非を 	 悟るだろう。
三平君に 	 至っては
水に 	 住む
山に 	 住む
三鞭酒を 	 御馳走して
得意と 	 思う
事が 	 出来れば
藤さんは 	 転がって行く。
どこまでも 	 転がって行く。
泥が 	 つく。
泥が 	 ついても
ものよりも 	 利く。
幅が 	 利く。
猫と 	 生れて
世に 	 住む
事 	 なる。
二年越しに 	 なる。
自分では 	 あるまいと
見識家は 	 あるまいと
先達て

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる